## Step 1: Getting ready of Spark and MongoDB instances

In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
from pymongo import MongoClient

In [10]:
# Initialize Spark Session
spark = SparkSession.builder \
    .appName("DataIngestion") \
    .getOrCreate()

# MongoDB connection string
mongo_uri = "mongodb://localhost:27017/myDatabase"
client = MongoClient(mongo_uri)
db = client['myDatabase']

## Step 2: Read first few lines of each dataset files

In [6]:
import pandas as pd
import json
import os
import xml.etree.ElementTree as ET

# Define the base path for the "demo_data" folder
base_path = os.path.join(os.getcwd(), "demo_data")

# Ensure the "demo_data" directory exists
os.makedirs(base_path, exist_ok=True)

# File paths
csv_file_path = os.path.join(base_path, "datafile.csv")
json_file_path = os.path.join(base_path, "datafile.json")
xml_file_path = os.path.join(base_path, "datafile.xml")
txt_file_path = os.path.join(base_path, "datafile.txt")

Generate demo dataset: 

- csv content
- json content
- xml content
- txt content

In [7]:
csv_content = """title,author,published
Moby Dick,Herman Melville,1851
Pride and Prejudice,Jane Austen,1813
1984,George Orwell,1949
"""

csv_file_path = os.path.join(base_path, "datafile.csv")
with open(csv_file_path, 'w') as file:
    file.write(csv_content)
    
json_content = [
    {"title": "Moby Dick", "author": "Herman Melville", "published": 1851},
    {"title": "Pride and Prejudice", "author": "Jane Austen", "published": 1813},
    {"title": "1984", "author": "George Orwell", "published": 1949}
]

json_file_path = os.path.join(base_path, "datafile.json")
with open(json_file_path, 'w') as file:
    json.dump(json_content, file, indent=4)

xml_content = """<books>
    <book>
        <title>Moby Dick</title>
        <author>Herman Melville</author>
        <published>1851</published>
    </book>
    <book>
        <title>Pride and Prejudice</title>
        <author>Jane Austen</author>
        <published>1813</published>
    </book>
    <book>
        <title>1984</title>
        <author>George Orwell</author>
        <published>1949</published>
    </book>
</books>
"""

xml_file_path = os.path.join(base_path, "datafile.xml")
with open(xml_file_path, 'w') as file:
    file.write(xml_content)

txt_content = """title\tauthor\tpublished
Moby Dick\tHerman Melville\t1851
Pride and Prejudice\tJane Austen\t1813
1984\tGeorge Orwell\t1949
"""

txt_file_path = os.path.join(base_path, "datafile.txt")
with open(txt_file_path, 'w') as file:
    file.write(txt_content)

In [25]:
# Step 2: Read first few lines of each dataset files

# Read a few lines from CSV
df_csv = spark.read.format("csv").option("header", "true").load(csv_file_path)
df_csv.show(5)

+-------------------+---------------+---------+
|              title|         author|published|
+-------------------+---------------+---------+
|          Moby Dick|Herman Melville|     1851|
|Pride and Prejudice|    Jane Austen|     1813|
|               1984|  George Orwell|     1949|
+-------------------+---------------+---------+



In [29]:
# Read the JSON file
with open(json_file_path, 'r') as file:
    data = json.load(file)

# Print the JSON data
# If it's a list of records, you might want to iterate through it
if isinstance(data, list):
    for record in data:
        print(record)
else:  # It's a single JSON object
    print(data)

{'title': 'Moby Dick', 'author': 'Herman Melville', 'published': 1851}
{'title': 'Pride and Prejudice', 'author': 'Jane Austen', 'published': 1813}
{'title': '1984', 'author': 'George Orwell', 'published': 1949}


In [32]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
import pyspark.sql.functions as F

# Initialize SparkSession (assuming it's not initialized)
spark = SparkSession.builder.appName("JsonExample").getOrCreate()

# Define the schema corresponding to the JSON data
schema = StructType([
    StructField("title", StringType(), True),
    StructField("author", StringType(), True),
    StructField("published", IntegerType(), True)
])

# Assuming json_file_path is defined and points to the correct JSON file
df_json = spark.read.schema(schema).json(json_file_path)
df_json.show()

+-----+------+---------+
|title|author|published|
+-----+------+---------+
| NULL|  NULL|     NULL|
| NULL|  NULL|     NULL|
| NULL|  NULL|     NULL|
| NULL|  NULL|     NULL|
| NULL|  NULL|     NULL|
| NULL|  NULL|     NULL|
| NULL|  NULL|     NULL|
| NULL|  NULL|     NULL|
| NULL|  NULL|     NULL|
| NULL|  NULL|     NULL|
| NULL|  NULL|     NULL|
| NULL|  NULL|     NULL|
| NULL|  NULL|     NULL|
| NULL|  NULL|     NULL|
| NULL|  NULL|     NULL|
| NULL|  NULL|     NULL|
| NULL|  NULL|     NULL|
+-----+------+---------+



In [33]:

# Read a few lines from XML
df_xml = spark.read.format('com.databricks.spark.xml').options(rowTag='your_row_tag').load(xml_file_path)
df_xml.show(5)

# Read a few lines from TXT tab-limited
df_txt = spark.read.option("delimiter", "\t").csv(txt_file_path)
df_txt.show(5)

Py4JJavaError: An error occurred while calling o247.load.
: org.apache.spark.SparkClassNotFoundException: [DATA_SOURCE_NOT_FOUND] Failed to find the data source: com.databricks.spark.xml. Please find packages at `https://spark.apache.org/third-party-projects.html`.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.dataSourceNotFoundError(QueryExecutionErrors.scala:724)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:647)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:697)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:208)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:186)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodA

## Step 3: Data population and Data fusion into 

In [34]:
# Convert Spark DataFrame to Pandas DataFrame and insert into MongoDB
pd_df_csv = df_csv.limit(5).toPandas()
db.csv_collection.insert_many(pd_df_csv.to_dict('records'))

pd_df_json = df_json.limit(5).toPandas()
db.json_collection.insert_many(pd_df_json.to_dict('records'))

pd_df_xml = df_xml.limit(5).toPandas()
db.xml_collection.insert_many(pd_df_xml.to_dict('records'))

pd_df_txt = df_txt.limit(5).toPandas()
db.txt_collection.insert_many(pd_df_txt.to_dict('records'))

print("Data has been populated into MongoDB collections.")

NameError: name 'df_xml' is not defined